In [1]:
%%capture
help('modules')
from pgljupyter import LsystemWidget
#from openalea.vmango.preparation.glm_estimation.get_data_for_glm import main as get_data_for_glm
#import rpy2.robjects as r
import os

In [2]:
%%capture
#os.chdir('../src/openalea/vmango/preparation/glm_estimation')
# generate input files located at glm_estimate_input/{cultivar} for prob. tables
#get_data_for_glm()

# generate prob. tables at glm_output_proba/{cultivar}
#rscript = 'estimate_nb_glm_order_1_proba.r'
#r.r(f"source('{rscript}')")

In [3]:
os.chdir('../src/openalea/vmango/simulation')

In [4]:
lw = LsystemWidget('mango_simulation.lpy', unit='cm', size_display=(800, 600), size_world=(5,5,5))

In [5]:
lw

LsystemWidget(derivationLength=15, scene={'data': b'x\xda\xcc\xbd\x07\\M\xff\x1f\xf8_B\xa9h\xd2\xa4\xa9\x94u\x…

In [6]:
lw.editor

ParameterEditor(children=(VBox(children=(HBox(children=(HBox(children=(Button(description='Apply changes', sty…